# XMLCorpus
Una herramienta personalizada para analizar datos
contenidos en ficheros `xml`.

La ejecución se realiza sobre un [Jupyter Notebook](https://jupyter.org/),
un sistema de ejecución que permite integrar las
herramientas necesarias para que el sistema funcione
sin mayor complejidad.

## Estructura del `XML`
Siguiendo de ejemplo el fichero `xml` provisto,
este programa está especialmente diseñado para funcionar
con la siguiente estructura:

```xml
<!-- Notation:
    x, y: mandatory values
    o: optional values
-->
<propiel>
    <annotation>
        <!-- Optional -->
        <parts-of-speech>
            <value tag="x" summary="y" />
                        ...
        </parts-of-speech>
        <!-- Mandatory -->
        <morphology>
            <field tag="x">
                <value tag="x" summary="y" />
                            ...
            </field>
            ...
        </morphology>
        <!-- Optional -->
        <gloss>
            <value tag="x" summary="y" />
        </gloss>
    </annotation>

    <source id="x" alignment-id="o" language="y">
        <!-- All these fields are optional -->
        <title></title>
        <citation-part></citation-part>
        <editorial-note></editorial-note>
        <annotator></annotator>
        <reviewer></reviewer>
        <electronic-text-original-url></electronic-text-original-url>
        <!-- Source main section -->
        <div>
            <title>Actually unused</title>
            <sentence id="x" alignment-id="o" status="[annotated, unannotated, reviewed]">
                <token id="x" form="y" alignment-id="o" lemma="o" part-of-speech="o" morphology="o" gloss="o" />
                ...
            </sentence>
            ...
        </div>
    </source>
    ...
</propiel>
```
## Ejecución de las celdas
Las celdas que contienen texto y/o código se pueden
ejecutar. Esto permite, por ejemplo, que se pueda
cambiar este texto y que se muestre actualizado así
como ejecutar código en particular.

Para ejecutar una celda, existen dos opciones:
 + Con la celda seleccionada, pulsar el botón
   `Run` que aparece en la barra de opciones.

 + Con la celda seleccionada, pulsar <kbd>⇧ Shift</kbd> + <kbd>↵ Enter</kbd>.

Cualquiera de las dos opciones anteriores es perfectamente
válida para ejecutar una celda.

## Instalación de requisitos
Para que la aplicación funcione correctamente, es
necesario instalar las siguientes dependencias:

In [ ]:
pip install -r requirements.txt

## Carga de los datos
Para poder evaluar distintos ficheros `XML` se ofrece
un sistema de carga que permite o bien trabajar con
una URL o bien con un fichero `XML` en sí.

En el *widget* que aparece justo debajo tienes la opción
de subir un fichero o trabajar con una URL. Ten en cuenta
que, si pones una URL, esta tendrá prioridad sobre el
fichero.

In [ ]:
from xmlcjupyter import init, show_widgets, load_source, annotations_widget

init()
uploader, url = show_widgets()

Además, se puede trabajar con dos ficheros `XML` por
separado: un primer fichero que contenga únicamente
las anotaciones y el segundo que contenga ya las
fuentes. En este caso, se marcaría la casilla
y se especificaría la ubicación de dicho fichero.

In [ ]:
use_annotations, file_data = annotations_widget()

In [ ]:
annotations_tree, tree = load_source(uploader, url, use_annotations, file_data)

## Procesado de los datos
Ahora el fichero `XML` está ya cargado y listo
para ser utilizado. El objeto `tree` contiene todos los
valores del mismo, pero ahora es necesario hacer que
el programa lo entienda.

La función
```python
def parse_tree(tree, annotations) -> Dict[SourceID, Source]
```
identifica, dentro del árbol `XML`, las distintas
fuentes (`<source>`) que lo componen y genera un mapa
con distintas claves y valores. La clave consta
del identificador de una fuente (por ejemplo, `text1`)
y el valor es la fuente en sí, con todas las oraciones,
*tokens* y demás valores pertinentes.

Para ello, primero se han de cargar las anotaciones en
sí. Para ello, se usa la siguiente función:

In [ ]:
from xmlcjupyter import load_annotations

annotations = load_annotations(annotations_tree)

# Show the identified annotations
print(annotations)

Después, ya se pueden obtener las fuentes:


In [ ]:
from xmlcjupyter import parse_tree

sources = parse_tree(tree, annotations)

# Show the identified sources
print("Identified sources:", end=' {')
for key in sources.keys():
    print(key, end=', ')
print("}\nAccess them by writting: 'sources['sourceID']'")

## Operaciones con las fuentes
Ahora que ya están las fuentes listas para trabajar con
ellas, se pueden realizar las siguientes operaciones:

 + Mostrar el contenido de las fuentes como tabla.
 Para ello, se cuenta con el método
 ```python
 def display_source(source, tabletype="latex")
 ```
 el cual muestra el contenido de la fuente o de todas
 las fuentes, en caso de usar `sources` como parámetro.

 + Comparar el contenido de dos fuentes, mediante
 el método
 ```python
 def compare(source, another_source, sentences=(), status=None, tabletype="latex")
 ```
 el cual mostrará en una tabla las oraciones junto con
 sus *token*s anotados y las relaciones entre ellos.

 + Buscar dentro de una fuente según ciertas
 características. Por ejemplo, si se quiere buscar
 por todos los *token*s en primera persona del singular,
 o si son adjetivos, etc. Para ello, se cuenta con
 el método:
 ```python
 def find_words_by(data, source, tabletype="latex")
 ```

Como se puede ver, los métodos reciben un parámetro
adicional `tabletype` que por defecto está con un
valor "latex". Esto permite cambiar cómo se muestran
las tablas y si se quiere otro formato (hay una lista
con todos los formatos disponible en https://github.com/astanin/python-tabulate#table-format).

A continuación, a modo explicativo, se muestran ejemplos
de cómo usar las funciones anteriores:

### Mostrar todas las fuentes con sus datos
```python
display_source(sources)
```

### Mostrar los datos de una fuente con ID `text1`
```python
display_source(sources['text1'])
```

### Mostrar los datos de una fuente con otro formato de tabla
```python
display_source(sources['text2'], tabletype="grid")
```

### Comparar `text1` con `text2`
```python
compare(sources['text1'], sources['text2'])
```

### Comparar `text1` con `text2` las oraciones '0a' y '1a'
```python
compare(sources['text1'], sources['text2'], sentences=('0a', '1a'))
```

### Comparar `text1` con `text2` aquellas oraciones que estén anotadas
```python
compare(sources['text1'], sources['text2'], status=AnnotationStatus.ANNOTATED)
```

### Buscar palabras según distintos campos
```python
source = sources['text1']

# Find all first person tokens in text1
find_words_by({
    AnnotationsElements.Morphology: 'person.1'
}, source)

# Find all first person singular masculine tokens in text1
find_words_by({
    AnnotationsElements.Morphology: {'person.1', 'number.s', 'gender.m'}
}, source)

# Find all first person singular adjectives
find_words_by({
    AnnotationElements.Morphology: {'person.1', 'number.s'},
    AnnotationElements.PartsOfSpeech: 'A-'
}, source)

# Find all words literally translated from Latin (in example)
find_words_by({
    AnnotationElements.Gloss: 'l'
}, sources['text2'])

# Find all positive words in all sources
find_words_by({
    AnnotationElements.Morphology: 'degree.p'
}, sources)
```

## Declaración de las cabezeras de las funciones
A continuación, se declaran las cabeceras de las
funciones anteriores

In [ ]:
from xmlcjupyter import (
    display_source,
    compare,
    find_words_by
)
from xmlc import AnnotationStatus, AnnotationElements

## Código de prueba (funcional con `lindgos.xml`)

In [ ]:
# Print all sources
display_source(sources, tabletype="fancy_grid")

print('Comparing text1 with text2 (all sentences):', end='\n\n')
# Compare all sentences
compare(sources['text1'], sources['text2'], tabletype="fancy_grid")

print('Comparing text1 with text2 (reviewed sentences):', end='\n\n')
# Compare all reviewed sentences
compare(sources['text1'], sources['text2'], status=AnnotationStatus.REVIEWED, tabletype="fancy_grid")

print('All first person words in text1:', end='\n\n')
# Find all first person words
find_words_by({
    AnnotationElements.Morphology: 'person.1'
}, sources['text1'], tabletype="fancy_grid")

print('All common nouns:', end='\n\n')
# Find all common nouns
find_words_by({
    AnnotationElements.PartsOfSpeech: 'Nb'
}, sources, tabletype="fancy_grid")

print('All first person singular masculine words in text1:', end='\n\n')
# Find all first person singular masculine words
find_words_by({
    AnnotationElements.Morphology: {'person.1', 'number.s', 'gender.m'}
}, sources['text1'], tabletype="fancy_grid")

print('All literal translated words:', end='\n\n')
# Find all literal translated words
find_words_by({
    AnnotationElements.Gloss: 'l'
}, sources['text2'], tabletype="fancy_grid")

print('All unassigned words:', end='\n\n')
# Find all unassigned words
find_words_by({
    AnnotationElements.PartsOfSpeech: 'X-'
}, sources, tabletype="fancy_grid")

## Código fuente
El código fuente de la aplicación al completo se
encuentra en: https://github.com/Javinator9889/XMLCorpus

Ahí se puede ver cómo está diseñado y qué opciones
ofrece, junto con la documentación de cada una de las
funciones que lo componen.

## Descarga de las salidas
Al estar trabajando sobre este entorno, todas las
salidas se guardan y pueden ser descargadas
posteriormente. Además, a continuación puedes seguir
creando más celdas para seguir trabajando, y una vez
acabado, descargar el resultado o ver cómo queda.

## $\LaTeX$
Actualmente, por comodidad, las tablas se muestran
en el formato "*fancy_grid*", pero pueden ser exportadas
fácilmente a $\LaTeX$ para usarlas posteriormente. Para
ello, en los comandos anteriores, bastaría con quitar
el campo "*tabletype*", ya que su valor por defecto
es $\LaTeX$.

In [ ]:
# Esta celda es para ti 😉
find_words_by({
    AnnotationElements.Morphology: {'number.s'},
    AnnotationElements.PartsOfSpeech: 'A-'
}, sources)